# Example 7: Performance Test (Single Thread)
Firstly, let's import some modules that we will depend on in the next examples.

In [1]:
import time
import random
import pickle
import threading
import seal
from seal import ChooserEvaluator, \
	Ciphertext, \
	Decryptor, \
	Encryptor, \
	EncryptionParameters, \
	Evaluator, \
	IntegerEncoder, \
	FractionalEncoder, \
	KeyGenerator, \
	MemoryPoolHandle, \
	Plaintext, \
	SEALContext, \
	EvaluationKeys, \
	GaloisKeys, \
	PolyCRTBuilder, \
	ChooserEncoder, \
	ChooserEvaluator, \
	ChooserPoly

In this thread we perform a timing of basic operations in single-threaded
execution. We use the following lambda function to run the timing example.

In [2]:
def performance_test_st(context):
    print_parameters(context)
    poly_modulus = context.poly_modulus()
    coeff_modulus = context.total_coeff_modulus()
    plain_modulus = context.plain_modulus()

    # Set up keys. For both relinearization and rotations we use a large
    # decomposition bit count for best possible computational performance.
    dbc = seal.dbc_max()
    print("Generating secret/public keys: ")
    keygen = KeyGenerator(context)
    print("Done")

    secret_key = keygen.secret_key()
    public_key = keygen.public_key()

    # Generate evaluation keys.
    ev_keys = EvaluationKeys()
    print("Generating evaluation keys (dbc = " + (str)(dbc) + "): ")
    time_start = time.time()
    keygen.generate_evaluation_keys(dbc, ev_keys)
    time_end = time.time()
    time_diff = time_end - time_start
    print("Done [" + (str)(1000000*time_diff) + " microseconds]")

    # Generate Galois keys. In larger examples the Galois keys can use
    # a significant amount of memory, which can become a problem in constrained
    # systems. The user should try enabling some of the larger runs of
    # the test (see below) and to observe their effect on the memory pool
    # allocation size. The key generation can also take a significant
    # amount of time, as can be observed from the print-out.
    gal_keys = GaloisKeys()
    """if not context.qualifiers().enable_batching:
        print("Given encryption parameters do not support batching.")
        return"""
    print("Generating Galois keys (dbc = " + (str)(dbc) + "): ")
    time_start = time.time()
    keygen.generate_galois_keys(dbc, gal_keys)
    time_end = time.time()
    time_diff = time_end - time_start
    print("Done [" + (str)(1000000*time_diff) + " microseconds]")

    encryptor = Encryptor(context, public_key)
    decryptor = Decryptor(context, secret_key)
    evaluator = Evaluator(context)
    crtbuilder = PolyCRTBuilder(context)
    encoder = IntegerEncoder(plain_modulus)

    # These will hold the total times used by each operation.
    time_batch_sum = 0
    time_unbatch_sum = 0
    time_encrypt_sum = 0
    time_decrypt_sum = 0
    time_add_sum = 0
    time_multiply_sum = 0
    time_multiply_plain_sum = 0
    time_square_sum = 0
    time_relinearize_sum = 0
    time_rotate_rows_one_step_sum = 0
    time_rotate_rows_random_sum = 0
    time_rotate_columns_sum = 0

    # How many times to run the test?
    count = 10

    # Populate a vector of values to batch.
    pod_vector = []
    for i in range(crtbuilder.slot_count()):
        pod_vector.append(random.randint(0, 4294967295) % plain_modulus.value())

    print("Running tests ")
    for i in range(count):
        # [Batching]
        # There is nothing unusual here. We batch our random plaintext matrix into
        # the polynomial. The user can try changing the decomposition bit count to
        # something smaller to see the effect. Note that the plaintext we use is of
        # the correct size, so no unnecessary reallocations are needed.
        plain = Plaintext(context.parms().poly_modulus().coeff_count(), 0)
        time_start = time.time()
        crtbuilder.compose(pod_vector, plain)
        time_end = time.time()
        time_batch_sum += (time_end - time_start)

        # [Unbatching]
        # We unbatch what we just batched.
        pod_vector2 = []
        time_start = time.time()
        decomposed_plain = plain
        crtbuilder.decompose(decomposed_plain)
        pod_vector2 = [decomposed_plain.coeff_at(i) for i in range(decomposed_plain.coeff_count())]
        time_end = time.time()
        time_unbatch_sum += (time_end - time_start)
        if pod_vector2 != pod_vector:
            print("Batch/unbatch failed. Something is wrong.")
            return

        # [Encryption]
        # We make sure our ciphertext is already allocated and large enough to
        # hold the encryption with these encryption parameters. We encrypt our
        # random batched matrix here.
        encrypted = Ciphertext(context.parms())
        time_start = time.time()
        encryptor.encrypt(plain, encrypted)
        time_end = time.time()
        time_encrypt_sum += (time_end - time_start)

        # [Decryption]
        # We decrypt what we just encrypted.
        plain2 = Plaintext(context.parms().poly_modulus().coeff_count(), 0)
        time_start = time.time()
        decryptor.decrypt(encrypted, plain2)
        time_end = time.time()
        time_decrypt_sum += (time_end - time_start)
        if plain2.to_string() != plain.to_string():
            print((str)(plain2.coeff_count()) + " " + (str)(plain2.significant_coeff_count()))
            print("Encrypt/decrypt failed. Something is wrong.")
            return

        # [Add]
        # We create two ciphertexts that are both of size 2, and perform a few
        # additions with them.
        encrypted1 = Ciphertext(context.parms())
        encryptor.encrypt(encoder.encode(i), encrypted1)
        encrypted2 = Ciphertext(context.parms())
        encryptor.encrypt(encoder.encode(i + 1), encrypted2)
        time_start = time.time()
        evaluator.add(encrypted1, encrypted1)
        evaluator.add(encrypted2, encrypted2)
        evaluator.add(encrypted1, encrypted2)
        time_end = time.time()
        time_add_sum += (time_end - time_start) / 3.0

        # [Multiply]
        # We multiply two ciphertexts of size 2. Since the size of the result
        # will be 3, and will overwrite the first argument, we reserve first
        # enough memory to avoid reallocating during multiplication.
        encrypted1.reserve(3)
        time_start = time.time()
        evaluator.multiply(encrypted1, encrypted2)
        time_end = time.time()
        time_multiply_sum += (time_end - time_start)

        # [Multiply Plain]
        # We multiply a ciphertext of size 2 with a random plaintext. Recall
        # that plain multiplication does not change the size of the ciphertext,
        # so we use encrypted2 here, which still has size 2.
        time_start = time.time()
        evaluator.multiply_plain(encrypted2, plain)
        time_end = time.time()
        time_multiply_plain_sum += (time_end - time_start)

        # [Square]
        # We continue to use the size 2 ciphertext encrypted2. Now we square it
        # which is a faster special case of homomorphic multiplication.
        time_start = time.time()
        evaluator.square(encrypted2)
        time_end = time.time()
        time_square_sum += (time_end - time_start)

        # [Relinearize]
        # Time to get back to encrypted1, which at this point is still of size 3.
        # We now relinearize it back to size 2. Since the allocation is currently
        # big enough to contain a ciphertext of size 3, no reallocation is needed
        # in the process.
        time_start = time.time()
        evaluator.relinearize(encrypted1, ev_keys)
        time_end = time.time()
        time_relinearize_sum += (time_end - time_start)

        # [Rotate Rows One Step]
        # We rotate matrix rows by one step left and measure the time.
        time_start = time.time()
        evaluator.rotate_rows(encrypted, 1, gal_keys)
        evaluator.rotate_rows(encrypted, -1, gal_keys)
        time_end = time.time()
        time_rotate_rows_one_step_sum += (time_end - time_start) / 2

        # [Rotate Rows Random]
        # We rotate matrix rows by a random number of steps. This is a bit more
        # expensive than rotating by just one step.
        row_size = crtbuilder.slot_count() / 2
        random_rotation = (int)((int)(random.random()*(row_size+1)) % row_size)
        time_start = time.time()
        evaluator.rotate_rows(encrypted, random_rotation, gal_keys)
        time_end = time.time()
        time_rotate_rows_random_sum += (time_end - time_start)

        # [Rotate Columns]
        # Nothing surprising here.
        time_start = time.time()
        evaluator.rotate_columns(encrypted, gal_keys)
        time_end = time.time()
        time_rotate_columns_sum += (time_end - time_start)

        # Print a dot to indicate progress.
        print(".")

    print(" Done")

    avg_batch = (time_batch_sum / count)*1000000
    avg_unbatch = (time_unbatch_sum / count)*1000000
    avg_encrypt = (time_encrypt_sum / count)*1000000
    avg_decrypt = (time_decrypt_sum / count)*1000000
    avg_add = (time_add_sum / count)*1000000
    avg_multiply = (time_multiply_sum / count)*1000000
    avg_multiply_plain = (time_multiply_plain_sum / count)*1000000
    avg_square = (time_square_sum / count)*1000000
    avg_relinearize = (time_relinearize_sum / count)*1000000
    avg_rotate_rows_one_step = (time_rotate_rows_one_step_sum / count)*1000000
    avg_rotate_rows_random = (time_rotate_rows_random_sum / count)*1000000
    avg_rotate_columns = (time_rotate_columns_sum / count)*1000000

    print("Average batch: " + (str)(avg_batch) + " microseconds")
    print("Average unbatch: " + (str)(avg_unbatch) + " microseconds")
    print("Average encrypt: " + (str)(avg_encrypt) + " microseconds")
    print("Average decrypt: " + (str)(avg_decrypt) + " microseconds")
    print("Average add: " + (str)(avg_add) + " microseconds")
    print("Average multiply: " + (str)(avg_multiply) + " microseconds")
    print("Average multiply plain: " + (str)(avg_multiply_plain) + " microseconds")
    print("Average square: " + (str)(avg_square) + " microseconds")
    print("Average relinearize: " + (str)(avg_relinearize) + " microseconds")
    print("Average rotate rows one step: " + (str)(avg_rotate_rows_one_step) + " microseconds")
    print("Average rotate rows random: " + (str)(avg_rotate_rows_random) + " microseconds")
    print("Average rotate columns: " + (str)(avg_rotate_columns) + " microseconds")


def print_parameters(context):
    print("/ Encryption parameters:")
    print("| poly_modulus: " + context.poly_modulus().to_string())
    print("| coeff_modulus_size: " + (str)(context.total_coeff_modulus().significant_bit_count()) + " bits")
    print("| plain_modulus: " + (str)(context.plain_modulus().value()))
    print("| noise_standard_deviation: " + (str)(context.noise_standard_deviation()))

In [3]:
parms = EncryptionParameters()
parms.set_poly_modulus("1x^4096 + 1")
parms.set_coeff_modulus(seal.coeff_modulus_128(4096))
parms.set_plain_modulus(786433)
context = SEALContext(parms)
performance_test_st(context)

/ Encryption parameters:
| poly_modulus: 1x^4096 + 1
| coeff_modulus_size: 110 bits
| plain_modulus: 786433
| noise_standard_deviation: 3.19
Generating secret/public keys: 
Done
Generating evaluation keys (dbc = 60): 
Done [7695.198059082031 microseconds]
Generating Galois keys (dbc = 60): 
Done [174469.7093963623 microseconds]
Running tests 
.
.
.
.
.
.
.
.
.
.
 Done
Average batch: 262.451171875 microseconds
Average unbatch: 1525.9742736816406 microseconds
Average encrypt: 4581.809043884277 microseconds
Average decrypt: 987.9589080810547 microseconds
Average add: 38.305918375651046 microseconds
Average multiply: 11007.928848266602 microseconds
Average multiply plain: 1163.9595031738281 microseconds
Average square: 8077.049255371094 microseconds
Average relinearize: 1212.930679321289 microseconds
Average rotate rows one step: 1247.704029083252 microseconds
Average rotate rows random: 4961.442947387695 microseconds
Average rotate columns: 1244.7834014892578 microseconds


In [4]:
parms.set_poly_modulus("1x^8192 + 1")
parms.set_coeff_modulus(seal.coeff_modulus_128(8192))
parms.set_plain_modulus(786433)
context = SEALContext(parms)
performance_test_st(context)

/ Encryption parameters:
| poly_modulus: 1x^8192 + 1
| coeff_modulus_size: 219 bits
| plain_modulus: 786433
| noise_standard_deviation: 3.19
Generating secret/public keys: 
Done
Generating evaluation keys (dbc = 60): 
Done [53236.72294616699 microseconds]
Generating Galois keys (dbc = 60): 
Done [1292744.3981170654 microseconds]
Running tests 
.
.
.
.
.
.
.
.
.
.
 Done
Average batch: 503.08704376220703 microseconds
Average unbatch: 2983.4985733032227 microseconds
Average encrypt: 10767.769813537598 microseconds
Average decrypt: 3614.211082458496 microseconds
Average add: 96.5436299641927 microseconds
Average multiply: 42946.26712799072 microseconds
Average multiply plain: 5271.8400955200195 microseconds
Average square: 31246.423721313477 microseconds
Average relinearize: 7272.887229919434 microseconds
Average rotate rows one step: 7404.26778793335 microseconds
Average rotate rows random: 35772.44281768799 microseconds
Average rotate columns: 7391.953468322754 microseconds


In [5]:
parms.set_poly_modulus("1x^16384 + 1")
parms.set_coeff_modulus(seal.coeff_modulus_128(16384))
parms.set_plain_modulus(786433)
context = SEALContext(parms)
# performance_test_st(context) # uncoment to run

In [6]:
parms.set_poly_modulus("1x^32768 + 1")
parms.set_coeff_modulus(seal.coeff_modulus_128(32768))
parms.set_plain_modulus(786433)
context = SEALContext(parms)
# performance_test_st(context) # uncoment to run